In [1]:
from flask import Flask, escape,request,jsonify,session
import datetime
import re
import requests
import logging
import json
import datetime
import secrets

In [2]:
import adaptive_test as adt

In [3]:
adt_test = adt.adaptiveTest(items='../data/Math/P1-2_processed.csv')
# adt_test.register_examinee('123','sdfasdfss')

In [4]:
# Flask server
app = Flask(__name__)
app.config['SECRET_KEY'] = 'top_secret123xxxyyy'
app.config['PERMANENT_SESSION_LIFETIME'] =  datetime.timedelta(minutes=10)

In [5]:

@app.route('/adaptivetest',methods=['GET','POST'])
def adaptive_test():
    if request.method=='POST':
        input_data = request.get_json()
        
        # register student for test session
        if input_data['command']=='new':
            if str(input_data['id']) in session:
                return jsonify({'id':input_data['id'],
                                'status' : 'session_error',
                                'message' : 'id already exist'
                               })
            
            temp = secrets.token_hex()
            status = adt_test.register_examinee(str(input_data['id']),temp)
            print(adt_test.active_examinees)
            if status:
                session[input_data['id']] = temp
                return jsonify({'id':input_data['id'],
                                'session_id': session[input_data['id']],
                                'status' : 'quiz_start'
                               })
            else:
                return jsonify({'id':input_data['id'],
                                'status' : 'session_error',
                                'message' : 'fail to register'
                               })
            
        # session error
        try:
            # session time out
            if input_data['id'] not in session:
                try:
                    status = adt_test.remove_examinee(input_data['id'],input_data['session_id'])
                    if status:
                        print('Session timeout')
                    print(adt_test.active_examinees)
                except:
                    return jsonify({'id':input_data['id'],
                                'status' : 'session_error',
                                'message': 'session time out'
                               })
            # session id check
            if (session[input_data['id']] != input_data['session_id']):
                return jsonify({'id':input_data['id'],
                                'status' : 'session_error',
                                'message' : 'session id incorrect'
                               })
        except:
            return jsonify({'id':input_data['id'],
                                'status' : 'session_error',
                            'message': 'pls register first'
                               })
        
        if input_data['command']=='end':
            status = adt_test.remove_examinee(input_data['id'],input_data['session_id'])
            print(adt_test.active_examinees)
            if status:
                return jsonify({'id':input_data['id'],
                                'session_id': input_data['session_id'],
                                'status' : 'quiz_forced_end'
                               })
            else:
                return jsonify({'id':input_data['id'],
                                'status' : 'session_error',
                                'message': 'check registeration'
                               })
            
        elif input_data['command']=='next':
            question = adt_test.get_question_index(input_data['id'],input_data['session_id'])
            print(adt_test.active_examinees)
            if question:
                if question==-1:
                    return jsonify({'id':input_data['id'],
                                'session_id': input_data['session_id'],
                                'status' : 'quiz_error',
                                'message' : 'submit answer first'
                               })
                else:
                    return json.dumps({'id':input_data['id'],
                                    'session_id': input_data['session_id'],
                                    'question_id': str(question['ID']),
                                    'question': question['Question'],
                                    'choices': {item:question[item] for item in ['A','B','C','D']},
                                    'correct_answer': question['Answer'],
                                    'status' : 'quiz_question',
                                   })
            else:
                return jsonify({'id':input_data['id'],
                                'session_id': input_data['session_id'],
                                'status' : 'quiz_end'
                               })
            
        elif input_data['command']=='answer':
            status = adt_test.update_response(input_data['id'],input_data['session_id'],input_data['chosen_answer'])
            print(adt_test.active_examinees)
            if status:
                return json.dumps({'id':input_data['id'],
                                'session_id': input_data['session_id'],
                                'question_id': str(input_data['question_id']),
                                'status':'quiz_ans_update'
                               })
            else:
                return json.dumps({'id':input_data['id'],
                                'session_id': input_data['session_id'],
                                'question_id': str(input_data['question_id']),
                                'status' : 'quiz_ans_update_fail'
                               })
                
            
            



In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Dec/2019 00:07:15] "POST /adaptivetest HTTP/1.1" 200 -


{'1232bcf45368f8f19f41f5acfeba9fe142defaa5d74c99ec090cd99336ebc0c56a0': {'start_time': datetime.datetime(2019, 12, 16, 0, 7, 15, 693594), 'exam_id': '1232bcf45368f8f19f41f5acfeba9fe142defaa5d74c99ec090cd99336ebc0c56a0', 'student_id': '123', 'session_id': '2bcf45368f8f19f41f5acfeba9fe142defaa5d74c99ec090cd99336ebc0c56a0', 'administered_items': [], 'responses': [], 'history': None, 'est_theta': 1.8418793623894478}}


127.0.0.1 - - [16/Dec/2019 00:07:32] "POST /adaptivetest HTTP/1.1" 200 -


{'1232bcf45368f8f19f41f5acfeba9fe142defaa5d74c99ec090cd99336ebc0c56a0': {'start_time': datetime.datetime(2019, 12, 16, 0, 7, 15, 693594), 'exam_id': '1232bcf45368f8f19f41f5acfeba9fe142defaa5d74c99ec090cd99336ebc0c56a0', 'student_id': '123', 'session_id': '2bcf45368f8f19f41f5acfeba9fe142defaa5d74c99ec090cd99336ebc0c56a0', 'administered_items': [175], 'responses': [], 'history': None, 'est_theta': 1.8418793623894478}}


...

{'12378c957e24171d582c9fbc021b90ac223f3d677c092d5ae0b6b0fd02f73548560': {'start_time': datetime.datetime(2019, 12, 16, 0, 11, 31, 926682), 'exam_id': '12378c957e24171d582c9fbc021b90ac223f3d677c092d5ae0b6b0fd02f73548560', 'student_id': '123', 'session_id': '78c957e24171d582c9fbc021b90ac223f3d677c092d5ae0b6b0fd02f73548560', 'administered_items': [105], 'responses': [True], 'history': None, 'est_theta': -2.617011351122116}}


127.0.0.1 - - [16/Dec/2019 00:16:50] "POST /adaptivetest HTTP/1.1" 200 -


{'12378c957e24171d582c9fbc021b90ac223f3d677c092d5ae0b6b0fd02f73548560': {'start_time': datetime.datetime(2019, 12, 16, 0, 11, 31, 926682), 'exam_id': '12378c957e24171d582c9fbc021b90ac223f3d677c092d5ae0b6b0fd02f73548560', 'student_id': '123', 'session_id': '78c957e24171d582c9fbc021b90ac223f3d677c092d5ae0b6b0fd02f73548560', 'administered_items': [105], 'responses': [True], 'history': None, 'est_theta': -2.617011351122116}}


127.0.0.1 - - [16/Dec/2019 00:17:53] "POST /adaptivetest HTTP/1.1" 200 -


{}
